## Custom Segmentation Algorithm Dev

Notebook to build segmentation implementation ([spheroid_cytometer.py](spheroid_cytometer.py)).

In [1]:
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from scipy import ndimage
from skimage import feature
from scipy import ndimage
from skimage import morphology
from skimage import filters
from skimage import img_as_float32
from cytokit.ops import cytometry as cytometer_op
from cytokit.ops import tile_generator, tile_crop
from cytokit import math as ck_math
from cytokit import config as ck_config
from itkwidgets import view
%matplotlib inline
%run spheroid_cytometer.py
%run logging.py

In [324]:
exp_name = '20190215-mc38-dmso-control'
#exp_dir, tile_index = 'XY07', 0
#exp_dir, tile_index = 'XY02', 0
#exp_dir, tile_index = 'XY03', 2 # 4 clumps together
exp_dir, tile_index = 'XY03', 5 # A great example of a cell and two clumps all at different elevations
#exp_dir, tile_index = 'XY03', 4 # 3 somewhat close spheroids
data_dir = osp.join(os.environ['CYTOKIT_DATA_DIR'], 'cytokit', 'mc38-spheroid', exp_name, exp_dir, 'raw')
config_dir = osp.join(os.environ['CYTOKIT_REPO_DIR'], 'pub', 'config', 'mc38-spheroid')
config = ck_config.load(config_dir)
os.environ['CYTOKIT_PATH_FORMATS'] = 'get_default_path_formats("1_' + exp_dir + '_{tile:05d}_Z{z:03d}_CH{channel:d}.tif")'
config.register_environment()

In [325]:
tile_gen = tile_generator.CytokitTileGenerator(config, data_dir, 0, tile_index)
tile_cropper = tile_crop.CytokitTileCrop(config)

In [ ]:
tile = tile_cropper.run(tile_gen.run())
#tile = tile_gen.run()
tile.shape, tile.dtype

In [ ]:
from skimage import transform
img_lg = tile[0, :, 0]
img = transform.rescale(img_lg, (1, .5, .5), multichannel=False, preserve_range=True, anti_aliasing=True, mode='reflect').astype(img_lg.dtype)
img.shape, img.dtype

### Debugging

In [292]:
#view(img)

In [293]:

# def flat_ball(r, rz):
#     return np.stack([
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#         morphology.disk(r),
#         np.pad(morphology.disk(rz), r-rz, 'constant'),
#     ])

# def prep(img):
#     img = ndimage.median_filter(img, size=(1, 3, 3))
#     img = img_as_float(img)
#     return img

# #gimage = standardize(img_prep, 16)
# #gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
# gimage = prep(img)

# gimage = gimage - ndimage.gaussian_filter(gimage, sigma=(.1, 1, 1), mode='reflect')
# #gimage = gimage - ndimage.gaussian_filter(gimage, sigma=(0, 16, 16), mode='reflect')
# gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
# gimage = ndimage.generic_gradient_magnitude(gimage, ndimage.sobel)
# print(gimage.min(), gimage.max(), len(np.unique(gimage)))
# gimage = ndimage.gaussian_filter(gimage, sigma=(.2, 2, 2), mode='constant')
# #gimage = np.stack([filters.sobel(gimage[i]) for i in range(gimage.shape[0])])
# gimage = exposure.rescale_intensity(gimage, out_range=(0, 1))
# print(gimage.min(), gimage.max(), len(np.unique(gimage)))

# #mimage = gimage > filters.threshold_otsu(gimage)
# mimage = gimage > filters.threshold_li(gimage)

# selem = ndimage.generate_binary_structure(3, 1)
# selem[0] = False
# selem[2] = False
# mimage = ndimage.binary_closing(mimage, structure=selem, iterations=1)
# #mimage = ndimage.binary_closing(mimage, structure=ndimage.generate_binary_structure(3, 3), iterations=3)
# #mimage = np.stack([morphology.binary_closing(mimage[i]) for i in range(mimage.shape[0])])
# mimage = ndimage.binary_fill_holes(mimage, structure=selem)
# mimage = morphology.remove_small_objects(mimage, min_size=512)
# limage, nlabel = ndimage.label(mimage)
# print('Num objects:', nlabel)

# dimage = ndimage.distance_transform_edt(mimage, sampling=(10, 1, 1))
# dimage = ndimage.gaussian_filter(dimage, sigma=(.5, 5, 5), mode='constant')

# pimage = feature.peak_local_max(dimage, exclude_border=False, min_distance=100, indices=False, labels=limage)
# pimage = ndimage.label(pimage)[0]
# #pimage = ndimage.binary_dilation(pimage, iterations=3)

# simage = segmentation.watershed(-dimage, pimage, mask=mimage)


In [ ]:
%run spheroid_cytometer.py
# img_seg = SpheroidCytometer.segment(None, img_lg, include_intermediate_results=True, 
#     rescale_factor=None, sigmas=(1, 2, 5), z_sample_factor=.1, min_object_size=1024, min_peak_dist=100)
img_seg = SpheroidCytometer.segment(None, img_lg, include_intermediate_results=True, 
    rescale_factor=.5, sigmas=(.5, 1.5, 1), z_sample_factor=.01, min_object_size=int(1024/8), min_peak_dist=50)
# img_seg = SpheroidCytometer.segment(None, img_lg, include_intermediate_results=True, 
#                                     rescale_factor=.5, sigmas=(1, 8, 4), z_sample_factor=.01, min_object_size=1024, min_peak_dist=25)
#img_seg = SpheroidCytometer.segment(None, img_lg, rescale_factor=None, sigmas=(1, 1, 16), z_sample_factor=.1, min_object_size=1024, min_peak_dist=150)
simage = img_seg[:, 0]
simage.shape, simage.dtype

In [322]:
view(img_seg[:, -1].astype(np.float32).copy())

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [279]:
view(
    img_as_float32(simage.copy()),
    size_limit_3d=[1024, 1024, 1024],
    #size_limit_2d=[1024, 1024]
)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

In [ ]:
img0 = img_lg
img1 = np.stack([segmentation.find_boundaries(img_seg[i, 0], mode='inner') for i in range(img_seg.shape[0])])
img2 = np.stack([segmentation.find_boundaries(img_seg[i, 1], mode='inner') for i in range(img_seg.shape[0])])

view(
    (img_as_float(img0) + (1+img_as_float(img1)) + (5+img_as_float(img2))).astype(np.float32),
    size_limit_3d=[1024, 1024, 1024],
    #size_limit_2d=[1024, 1024]
)

In [206]:
plt.imshow((img_bound.astype(np.float32) * .5 + img_as_float(img).astype(np.float32))[13])
plt.gcf().set_size_inches(24, 24)

NameError: name 'img_bound' is not defined

### Pipeline Dev

In [5]:
op = cytometer_op.Cytometry2D(config)

In [6]:
op.initialize()

In [7]:
# %%time
# img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=None, min_object_size=1024, min_peak_dist=200)
# img_seg.shape

In [8]:
%%time
img_seg = op.cytometer.segment(tile[0, :, 0], rescale_factor=.5, min_object_size=512, min_peak_dist=75)
img_seg.shape

CPU times: user 1min 6s, sys: 11.7 s, total: 1min 17s
Wall time: 1min 17s


In [9]:
#view(img_seg[:, 5].astype(np.float32).copy())

In [10]:
%%time
stats = op.cytometer.quantify(tile, img_seg, sigma=(.1, 1, 1), channel_names=config.channel_names, 
                              cell_graph=True, morphology_features=False, 
                              cell_intensity=['mean', 'sum'])

CPU times: user 4min 4s, sys: 3.28 s, total: 4min 7s
Wall time: 4min 7s


In [11]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 26 columns):
id                     371 non-null int64
x                      371 non-null float64
y                      371 non-null float64
z                      371 non-null int64
cm:size                371 non-null int64
cm:diameter            371 non-null float64
cm:perimeter           371 non-null float64
cm:solidity            371 non-null float64
cm:circularity         371 non-null float64
nm:size                371 non-null int64
nm:diameter            371 non-null float64
nm:perimeter           371 non-null float64
nm:solidity            371 non-null float64
nm:circularity         371 non-null float64
cg:n_neighbors         371 non-null int64
cg:neighbor_ids        371 non-null object
cg:adj_neighbor_pct    371 non-null object
cg:adj_bg_pct          371 non-null object
ci:BF:mean             371 non-null float64
ci:LIVE:mean           371 non-null float64
ci:DEAD:mean           371 n

In [12]:
stats['id'].nunique()

23